# using cloudbridge to access openstack on Jetstream
This notebook demostrates the use of the cloudbridge api to add files to buckes, list the available vm images, create networks and routers and how to create an instance.  (The last step is not working completely.  we will fix that one later.)

If you do not have cloudbridge installed uncomment the next line and run it.

In [54]:
#!pip install cloudbridge --upgrade

In [1]:

from cloudbridge.cloud.factory import CloudProviderFactory, ProviderList


you will want to log into the open stack portal for jetspeed to get you project name.  

In [10]:
js_config = {"os_username": "your user name",
             "os_password": "and password",
             "os_auth_url": "https://jblb.jetstream-cloud.org:35357/v3",
             "os_user_domain_name": "tacc",
             "os_project_domain_name": "tacc",
             "os_project_name": "your project (see the portal)"}
js = CloudProviderFactory().create_provider(ProviderList.OPENSTACK, js_config)


we now have a handle to jetspeed!  let's look at a list of the public vm images.

In [11]:
js.compute.images.list()

[<CB-OpenStackMachineImage: ChromoSnapShot_b370dc51-2c95-496d-b4aa-19b52860863c_2017-01-30_09:44:41 (3153ba66-d60c-4591-859a-4786b96ff614)>,
 <CB-OpenStackMachineImage: R with Intel compilers (CentOS 7) (c112d8aa-31a1-4535-9c86-cd0decc3b1ce)>,
 <CB-OpenStackMachineImage: Myers L533 1st Image (dc7ce197-8217-4ff2-8e88-843a8094e427)>,
 <CB-OpenStackMachineImage: Myers L533 1st Image (f7a04d77-b267-4bef-93ba-b5b89b7f9e22)>,
 <CB-OpenStackMachineImage: Myers L533 1st Image (988a7951-3db1-4362-a29c-d19bcd65e89a)>,
 <CB-OpenStackMachineImage: Myers L533 1st Image (6b9faa14-a3dc-4e44-90dd-30dae0c33d93)>,
 <CB-OpenStackMachineImage: Intel Development (CentOS 7) (b5bfb513-1d79-49ff-aed0-b55c8fe0114b)>,
 <CB-OpenStackMachineImage: Chemcompute Slave Slurm (e4b7aa4e-1f13-4426-9b1b-152651114351)>,
 <CB-OpenStackMachineImage: Python3_Jupyter_Tensorflow (78e3a5c6-3aef-4030-b9b7-9899393643fe)>,
 <CB-OpenStackMachineImage: Centos 7 (7.2) Development GUI (fd3eafe8-7f07-4028-a87e-1b47ff558653)>,
 <CB-Open

the follow will list our current buckets and add a new one.

In [12]:
print js.object_store.list()
newbuck = js.object_store.create('my_new_bucket')
print(newbuck.name)

[<CB-OpenStackBucket: afourth_buck>, <CB-OpenStackBucket: escience>, <CB-OpenStackBucket: my_buck>, <CB-OpenStackBucket: my_buck2>, <CB-OpenStackBucket: my_bucket>, <CB-OpenStackBucket: my_new_buck>, <CB-OpenStackBucket: my_new_bucket>, <CB-OpenStackBucket: my_new_bucket2>, <CB-OpenStackBucket: my_new_bucket3>, <CB-OpenStackBucket: my_new_bucket4>]
my_new_bucket


let's get a handle to another new bucket

In [13]:
bucket = js.object_store.create('my_new_bucket2')

In [14]:
print bucket

<CB-OpenStackBucket: my_new_bucket2>


we can search for a bucket by name

In [15]:
buckes = js.object_store.find(name='my_new_bucket2')

In [16]:
buckes

[<CB-OpenStackBucket: my_new_bucket2>]

Now lets's add some stuff to a blob we will call 'stuff'

In [17]:
buckobj = buckes[0].create_object('stuff')

now we upload a tiny file called 'stuf' it contains 2 lines
<pre>
here is a file
of stuff
</pre>

In [18]:
fo = open('c:\users\dennis\documents\stuff.txt','rb')

In [19]:
buckobj.upload(fo)

In [20]:
js.object_store.list()

[<CB-OpenStackBucket: afourth_buck>,
 <CB-OpenStackBucket: escience>,
 <CB-OpenStackBucket: my_buck>,
 <CB-OpenStackBucket: my_buck2>,
 <CB-OpenStackBucket: my_bucket>,
 <CB-OpenStackBucket: my_new_buck>,
 <CB-OpenStackBucket: my_new_bucket>,
 <CB-OpenStackBucket: my_new_bucket2>,
 <CB-OpenStackBucket: my_new_bucket3>,
 <CB-OpenStackBucket: my_new_bucket4>]

In [21]:
buckes[0].list()

[<CB-OpenStackBucketObject: stuff>]

In [22]:
stuff = buckes[0].get('stuff')

In [23]:
itr = stuff.iter_content()

In [24]:
for it in itr:
    print it

here is a file
of stuff



now let's see if we can start up an image.  This has a bug so it doesn't work, but this shows what is supposed to work.   We will update when the bug is fixed.

In [25]:
img = js.compute.images.get('9e04c7e2-8527-4355-870a-ce825dbdcf8d')

In [26]:
img

<CB-OpenStackMachineImage: Ubuntu 14.04 with Hadoop 2.7.3 (9e04c7e2-8527-4355-870a-ce825dbdcf8d)>

In [27]:
inst_type = sorted([t for t in js.compute.instance_types.list()
                    if t.vcpus >= 4 and t.ram >= 8],
                   key=lambda x: x.vcpus*x.ram)[0]

In [28]:
inst_type

<CB-OpenStackInstanceType: m1.medium (3)>

the following is how to create a key-pair.  if it already exists you can grab it with the find() function.

In [64]:
#kp = js.security.key_pairs.create('cloudbridge_sec')
#with open('cloudbridge_sec.pem', 'w') as f:
#    f.write(kp.material)
#import os
#os.chmod('cloudbridge_sec.pem', 0400)
kp = js.security.key_pairs.find(name='cloudbridge_sec')

In [32]:
kp

[<CBKeyPair: cloudbridge_sec>]

In [33]:
sg = js.security.security_groups.list()[0]

In [34]:
sg

<CB-OpenStackSecurityGroup: 1eeae9e0-bf15-4089-9d23-9931625f8711>

In [43]:
netlist = js.network.list()

In [44]:
netlist

[<CB-OpenStackNetwork: 4367cd20-722f-4dc2-97e8-90d98c25f12e (public)>,
 <CB-OpenStackNetwork: 0e04d98d-f8c6-4882-9e1a-fd321b0414da (genapp-net)>,
 <CB-OpenStackNetwork: 4d7cc5c5-1be2-4af3-a208-63a4cd538903 (scicloud)>,
 <CB-OpenStackNetwork: 1ae6a914-9003-4176-a2f0-57e1bb31c52e (scicloud2)>]

In [37]:
net = js.network.get('4d7cc5c5-1be2-4af3-a208-63a4cd538903')

In [38]:
net

<CB-OpenStackNetwork: 4d7cc5c5-1be2-4af3-a208-63a4cd538903 (scicloud)>

In [42]:
net = js.network.create('scicloud2')
sn = net.create_subnet('10.0.0.0/28', 'scicloud2')
router = js.network.create_router('scicloud2')


In [61]:
if not net.external:
    for n in js.network.list():
        if n.external:
            external_net = n
            break
router.attach_network(external_net.id)
router.add_route(sn.id)

oops!  the following doesn't work.   

In [63]:
#inst = js.compute.instances.create( 
#    name='scicloud-VM', image=img, instance_type=inst_type,
#    key_pair=kp, security_groups=[sg], network=net)